## Get variable names and descriptions from BRFSS codebooks

In [518]:
import pandas as pd
import pickle

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import re
import PyPDF2
import pickle

from progress_bar import log_progress
from functools import reduce

First, let's load one of the CSVs and take a look at the data structure.

In [3]:
brfss_2014 = pd.read_csv("../data/brfss/brfss2014.csv", encoding = "cp1252", nrows=100)

In [5]:
brfss_2014.head()

,Unnamed: 0,x.state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,x.psu,...,x.fobtfs,x.crcrec,x.aidtst3,x.impeduc,x.impmrtl,x.imphome,rcsbrac1,rcsrace1,rchisla1,rcsbirth
0,1,1,1,1172014,1,17,2014,1100,2014000001,2014000001,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN
1,2,1,1,1072014,1,7,2014,1100,2014000002,2014000002,...,2.0,2.0,2.0,4,1,1,NaN,NaN,NaN,NaN
2,3,1,1,1092014,1,9,2014,1100,2014000003,2014000003,...,2.0,2.0,2.0,6,1,1,NaN,NaN,NaN,NaN
3,4,1,1,1072014,1,7,2014,1100,2014000004,2014000004,...,2.0,1.0,2.0,6,3,1,NaN,NaN,NaN,NaN
4,5,1,1,1162014,1,16,2014,1100,2014000005,2014000005,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN


## Extracting relevant predictors from BRFSS 1999-2016

BRFSS has a lot different variables, and unfortunately the name of the variables changes over the years. We need to extract the names of the variables and the description of each variable from the codebooks for each year. This is going to involve reading in each of the codebook pdf files and extracting the variable names and the description of the question that was asked for each variable. ONce we have a dictionary for the variable names and the questions, we can figure out which variables we want to get from each BRFSS CSV file.

In [5]:
def extract_pdf_string(pdf_path):
    
    pdfFileObj = open(pdf_path, 'rb')
    
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    num_pages =  pdfReader.numPages
    
    pdf_string = ""
    
    for n in range(num_pages):
        pageObj = pdfReader.getPage(n)
        page_string = pageObj.extractText()
        clean_page_string = re.sub("\s+", " ", page_string)
        clean_page_string = clean_page_string.strip()
        pdf_string += clean_page_string
        
    # closing the pdf file object
    pdfFileObj.close()
    return pdf_string

In [223]:
def find_var(words):
    for word in words:
        if word == 'BRFSS' or word == 'SAS' or len(word) <3:
            pass
        elif word[0] == '_':
            return word
        else:
            ratio = sum([letter.isupper() for letter in list(word)])/len(list(word))
            if ratio > 0.6:
                return word

In [227]:
def extract_variable_name_and_description(section):
    words = section.split(" ")
    var_name = find_var(words)
    
    if 'Description:' in section:
        subsections = section.split("Description:")
    elif 'Question:' in section:
        subsections = section.split("Question:")
    else:
        description = None
        return var_name, description
        
    description = subsections[1]

    value_limit = description.find("Value")
    weighted_limit = description.find("Weighted")

    if value_limit == -1:
        limit = weighted_limit
    elif weighted_limit == -1:
        limit = value_limit
    else:
        limit = min(description.find("Value"), description.find("Weighted"))

    description = description[0:limit]
    description = description.strip()

    if description == '':
        description = None
    
    return var_name, description

In [505]:
years = range(1999, 2018)

In [228]:
codebook_dfs_dict = {}

for year in log_progress(years):
    pdf_string = extract_pdf_string(f"../data/brfss/codebooks/{year}_codebook.pdf")
    pdf_sections = pdf_string.split("SAS Variable")

    var_desc_array = []
    for section in pdf_sections:
        row = extract_variable_name_and_description(section)
        var_desc_array.append(row)

    var_desc_df = pd.DataFrame(var_desc_array, columns=['var_name', 'description'])
    
    # We'll drop the rows where the variable is none, and then also get rid of duplicate rows where the var_names
    # are the same.
    var_desc_df = var_desc_df.dropna(subset=['var_name']).drop_duplicates(subset=['var_name'])

    codebook_dfs_dict[year] = var_desc_df

In [229]:
with open("../data/pickles/codebook_dfs_dict.pkl", "wb") as f:
    pickle.dump(codebook_dfs_dict, f)

We now have a dictionary of codebook dataframes for each year; the codebook dataframe has a column 'var_name' and a column 'description', and each row is a different variable. Now, let's try joining these dataframes together and seeing how it goes. The codebooks have different variable names for the same information between different years, which unfortunately means that in order to see how these names have evolved we have to use an outer join to get master dataframe of all of the variable names and their descriptions over the years.

In [232]:
master_df = codebook_dfs_dict[1999]
master_df.rename(columns={'description':1999}, inplace=True)
for year, df in codebook_dfs_dict.items():
    if year == 1999:
        pass
    else:
        master_df = pd.merge(master_df, df, how='outer', on='var_name')
        master_df.rename(columns={'description':year}, inplace=True)

In [516]:
with open("../data/pickles/master_codebook_all_years.pkl", "wb") as f:
    pickle.dump(master_df, f)

In [510]:
master_df.shape

(1572, 20)

In [508]:
master_df[master_df.var_name=='_LLCPWT']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1189,_LLCPWT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...


In [304]:
master_df[master_df.var_name=='_FINALWT']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
243,_FINALWT,None,None,Final weight assigned to each respondent.,Final weight assigned to each respondent,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We could do a merge with an inner join, which would not capture every variable name that was ever used over all the codebooks (which an outer join does).

In [235]:
intersection_df = codebook_dfs_dict[1999]
intersection_df.rename(columns={'description':1999}, inplace=True)
for year, df in codebook_dfs_dict.items():
    if year == 1999:
        pass
    else:
        intersection_df = pd.merge(master_df, df, on='var_name')
        intersection_df.rename(columns={'description':year}, inplace=True)

In [322]:
intersection_df.head()

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017
0,_STATE,None,None,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,...,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code
1,_PSU,None,None,None,Annual Sequence Number,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...
2,IDATE,None,None,None,None,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,...,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date
3,IMONTH,None,None,Month of interview,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,...,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month
4,IDAY,None,None,Calendar date of interview,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,...,Interview Day,Inte rview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day


We can keep only the variables where 5 or fewer codebooks did not have that variable; that is, most of the codebooks contain that variable name.

In [335]:
majority_vars = intersection_df[intersection_df.isna().sum(axis=1) <= 5]

We want to translate the master dataframe of all the variable names and their descriptions (with each year as a different column) into a dictionary. What we're interested in is the variable name as a key, and the most 'common' description as the value. The description/question wording changes between years, so we'll pick the description/question that has been used most repeatedly over the years.

In [330]:
def make_consensus_var_desc_dict(df):
    raw_dict = dict(pd.DataFrame.transpose(df.set_index('var_name')))
    clean_dict = {}
    for key, value in raw_dict.items():
        if value.mode().empty:
            clean_dict[key] = None
        else:
            clean_dict[key] = value.mode()[0]
    return clean_dict

It actually makes the most sense to make a consensus variable/description dictionary from the dataframe generated by merging with outer join, because then we can look through the key/value pairs and figure out which variables are synonyms with each other.

In [512]:
consensus_var_desc_dict = make_consensus_var_desc_dict(master_df)

In [514]:
with open("../data/pickles/consensus_var_desc_dict.pkl", "wb") as f:
    pickle.dump(consensus_var_desc_dict, f)

In [344]:
for key, value in d.items():
    if 'DIABE' in key:
        print(key)
        print(value)

DIABETES
5.1. [People may] provide regular care or assistance to [someone] who is elderly or has a long -term illness or di sability. During the past month, did you provide any such care or assistance to a family member or friend who is 60+ years of age? Column: 86
DIABEYE
Has a doctor ever told you that diabetes has affected your eyes or that you had retinopathy?
DIABEDU
Have you ever taken a course or class in how to manage your diabetes yourself?
DIABETE2
Have you ever been told by a doctor that you have diabetes (If "Yes" and respondent is female, ask "Was this only when you were pregnant?". If Respondent says pre -diabetes or borderline diabetes, use response code 4.)
DIABETE3
(Ever told) you have diabetes (If "Yes" and respondent is female, ask "Was this only when you were pregnant?". If Respondent says pre -diabetes or borderline diabetes, use response code 4.)


In [345]:
master_df[master_df.var_name.str.contains("DIABET") == True]

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
40,DIABETES,6.1. How long has it been since you last visit...,5.1. [People may] provide regular care or assi...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,DIABETE2,NaN,NaN,NaN,NaN,NaN,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,DIABETE3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...",(Ever told) you have diabetes (If ´Yes´ and re...,(Ever told) you have diabetes (If ´Yes´ and re...


## Reading brfss CSVs and extracting relevant variables.

In [347]:
brfss_1999_df = pd.read_csv("../data/brfss/csv/brfss1999.csv")

In [348]:
brfss_1999_df.head()

,Unnamed: 0,x.state,x.geostr,x.denstr,x.psu,x.record,idate,imonth,iday,iyear,...,x.rfdracu,x.rfdrchr,x.rfdrdri,x.raw,x.csa,x.wt1,x.poststr,x.finalwt,exerdis1,exerdis2
0,1,1,1,1,10290,1,1201999,1,20,1999,...,1,1,1,2.0,1.0,2.0,709.955703,1419.911405,NaN,NaN
1,2,1,1,1,10992,1,1131999,1,13,1999,...,1,1,1,2.0,1.0,2.0,919.545794,1839.091587,NaN,NaN
2,3,1,1,1,20586,1,2271999,2,27,1999,...,1,1,1,1.0,1.0,1.0,1024.247333,1024.247333,NaN,NaN
3,4,1,1,1,20667,1,2061999,2,6,1999,...,2,1,1,2.0,1.0,2.0,1190.937133,2381.874267,NaN,NaN
4,5,1,1,1,20813,1,2111999,2,11,1999,...,1,1,1,2.0,1.0,2.0,1190.937133,2381.874267,NaN,NaN


In [361]:
diabetes_columns = [col for col in brfss_1999_df.columns if 'diabe' in col]

In [362]:
diabetes_columns

['diabetes']

In [354]:
for col in brfss_1999_df.columns:
    if 'diabe' in col:
        z = col

In [422]:
columns_to_extract = ['FINALWT', 'DIABET', 'LLCPWT', 'MEDCOST', 'CHOLC']

In [449]:
columns_to_extract = ['SMOKE', 'FINALWT', 'LLCPWT']

In [477]:
columns_to_extract = ['CHOLC', 'TOLDHI2', 'FINALWT', 'LLCPWT']

In [486]:
columns_to_extract = ['LASTCHNG', 'ASTHMA2', 'ASTHMA3', 'FINALWT', 'LLCPWT']

In [490]:
columns_to_extract = ['STATE', 'EXERANY', 'FINALWT', 'LLCPWT']

In [423]:
for col in columns_to_extract:
    print(col.lower())

finalwt
diabet
llcpwt
medcost
cholc


In [438]:
def find_cols_this_df_has(list_of_columns, df):
    good_cols = []
    for col in df.columns:
        temp_col = col.replace("x.", "")
        for col_to_extract in list_of_columns:
            if col_to_extract.lower() in temp_col.lower():
                good_cols.append(col)
    return good_cols

In [439]:
find_cols_this_df_has(columns_to_extract, brfss_1999_df)

['smoke100', 'smokeday', 'smokenum', 'x.smoker2']

In [460]:
d['CHOLC']

'4.3. Have you ever been told by a doctor or other health professional that your blood cholesterol is high? Column: 99'

In [493]:
# Okay, let's go basic. Let's decide what we want to extract from each csv. How about let's extract the variable
# smoke100 and the weight from every csv.
list_of_temp_dfs = []

for year in log_progress(years):
    brfss_df = pd.read_csv(f"../data/brfss/csv/brfss{year}.csv", encoding="cp1252")#, nrows=10)
    cols = find_cols_this_df_has(columns_to_extract, brfss_df)
#     print(year)
#     print(cols)
    temp_df = brfss_df[cols].copy()
    
    temp_df['year'] = year
    list_of_temp_dfs.append(temp_df)

In [494]:
with open("../data/pickles/any_exercise_list_of_dfs.pkl", "wb") as f:
    pickle.dump(list_of_temp_dfs, f)

In [500]:
print(list_of_temp_dfs[0].exerany.isna().sum())
print(len(list_of_temp_dfs[0].exerany))
list_of_temp_dfs[0].exerany.value_counts()

130945
159989


1.0    20429
2.0     8148
9.0      452
7.0       15
Name: exerany, dtype: int64